In [1]:
import os
import json

from credentials import OPENAPI_KEY
os.environ["OPENAI_API_KEY"] = OPENAPI_KEY

import openai
from tqdm import tqdm
import pandas as pd

In [2]:
client = openai.OpenAI()

In [3]:
ROOT_PATH = "PATH"
ds_name = "17bintweet"
ds_path = f"{ROOT_PATH}/{ds_name}.jsonl"
with open(ds_path, "r") as f:
    my_dataset = [json.loads(line) for line in f]

In [4]:
my_dataset[0]

{'messages': [{'role': 'system',
   'content': 'VRL-GPT-4o Türkçe metinlerin duygusunu verebilen bir chatbottur. (Pozitif, Nötr, Negatif)'},
  {'role': 'user',
   'content': 'Bu Türkçe metin hangi duyguyu içeriyor?: "Ulan Wifi\'ye bağlıyım ben. Ona bağlıyken Turkcell internet paketin bitti diye nasıl mesaj atabilir bana ya? Onu da mı ödeyelim"? (Pozitif/Nötr/Negatif)'},
  {'role': 'assistant', 'content': 'Negatif'}]}

In [5]:
preds = []
refs  = []

for example in tqdm(my_dataset):
    response = client.chat.completions.create(
        model="gpt-4o",  # or "gpt-4-turbo"
        messages=example["messages"][:2]
    )
    
    model_output = response.choices[0].message.content
    ground_truth = example["messages"][-1]["content"]
    preds.append(model_output)
    refs.append(ground_truth)

  0%|          | 0/17289 [00:00<?, ?it/s]

100%|██████████| 17289/17289 [3:06:17<00:00,  1.55it/s]   


In [6]:
output_dir = "/cta/users/anajafi/chatGPT/GPT4o/outputs"
output_df = pd.DataFrame({"preds": preds, "refs": refs})
output_df.to_csv(f"{output_dir}/gpt4o-{ds_name}.csv", index=False)

In [7]:
output_df["preds"].value_counts()

preds
Negatif                                                                                                                                                                               4845
Negatif.                                                                                                                                                                              2881
Pozitif.                                                                                                                                                                              2315
Nötr                                                                                                                                                                                  2149
Pozitif                                                                                                                                                                               2067
                                                           